In [1]:
import numpy
from sklearn.linear_model import LinearRegression
from sqlalchemy import between

In [2]:
from db import session_maker
from models import BusPos
session = session_maker()

2020-04-07 16:06:49,528 - katana - INFO - Using development settings


In [3]:
lower_bound = 1585387487
upper_bound = 1585391087 # 1 hour worth of data

bus_positions = session.query(BusPos).filter(
    BusPos.bus_internal_id == 3885,
    BusPos.bus_line_id == 70,
    BusPos.timestamp.between(lower_bound, upper_bound)
).all()

In [4]:
import json
import geojson

coords = [(float(x.lon), float(x.lat)) for x in bus_positions]
json.dumps(geojson.MultiLineString(coords))

'{"type": "MultiLineString", "coordinates": [[-64.11435, -31.42766], [-64.11435, -31.427655], [-64.114351, -31.427654], [-64.114352, -31.427652], [-64.116029, -31.423578], [-64.128932, -31.424539], [-64.128888, -31.423488], [-64.132169, -31.422468], [-64.13369, -31.422454], [-64.139719, -31.421931], [-64.142921, -31.422673], [-64.143173, -31.423796], [-64.147973, -31.42291], [-64.148804, -31.423275], [-64.150213, -31.423181], [-64.153036, -31.422546], [-64.155121, -31.422716], [-64.156823, -31.422933], [-64.158466, -31.423555], [-64.159977, -31.423388], [-64.163257, -31.423125], [-64.164657, -31.423313], [-64.173011, -31.42409], [-64.174735, -31.422174], [-64.175558, -31.420436], [-64.175874, -31.419702], [-64.178079, -31.419046], [-64.179129, -31.418733], [-64.184358, -31.417081], [-64.187148, -31.41647], [-64.187461, -31.416258], [-64.188391, -31.415977], [-64.197614, -31.4124], [-64.197724, -31.409414], [-64.199285, -31.408925], [-64.202687, -31.407895], [-64.203724, -31.40758], [-6

In [5]:
from geopy.distance import geodesic
from core.route import Route

route_handler = Route(session, 70)
first_route_id = route_handler.routes[0]['route_id']
first_route_description = route_handler.routes[0]['route_description']
print("Using route id {} {}".format(first_route_id, first_route_description))
first_route = route_handler.get_route_positions(first_route_id)[0]

first_route_pos = (
    float(first_route['lat']),
    float(first_route['lon'])
)

distances = []

for bus_pos in bus_positions:
    last_pos = (
        float(bus_pos.lat),
        float(bus_pos.lon)
    )
    
    distances.append(
        geodesic(first_route_pos, last_pos)
    )

Using route id 1691 HACIA B° VALLE ESCONDIDO


In [5]:
x = numpy.array([x.timestamp - bus_positions[0].timestamp for x in bus_positions]).reshape((-1, 1))
#y = numpy.array([x.meters for x in distances]).reshape((-1, 1))
y = numpy.array([15478.778287832372,
 15513.784656366914,
 15717.112591394995,
 15900.96928269953,
 15991.16640119209,
 15966.918794584251,
 15872.537279148073,
 15574.456379705123,
 15376.210000940628]).reshape((-1, 1))

for i in range(len(y)):
    print("({}, {})".format(x.item(i), y.item(i)))

(0, 15478.778287832372)
(95, 15513.784656366914)
(128, 15717.112591394995)
(190, 15900.96928269953)
(477, 15991.16640119209)
(668, 15966.918794584251)
(700, 15872.537279148073)
(765, 15574.456379705123)
(796, 15376.210000940628)


In [6]:
model = LinearRegression().fit(x, y)
r_sq = model.score(x, y)

ValueError: Found input variables with inconsistent numbers of samples: [48, 9]

In [8]:
print('intercept:', model.intercept_)
print('slope:', model.coef_)

print("y = {}*x + {}".format(model.coef_.item(0), model.intercept_.item(0)))

intercept: [-1470.81633966]
slope: [[4.69979004]]
y = 4.699790037317242*x + -1470.8163396628624
